In [1]:
# source /opt/envvars.sh

!pip3 install python-dotenv
%load_ext dotenv
%dotenv -o /opt/envvars.sh
%env

  Using cached https://files.pythonhosted.org/packages/32/2e/e4585559237787966aad0f8fd0fc31df1c4c9eb0e62de458c5b6cde954eb/python_dotenv-0.15.0-py2.py3-none-any.whl


{'HOSTNAME': 'hadoop',
 'OLDPWD': '/',
 'PWD': '/opt',
 'HOME': '/home/hadoop',
 'SHELL': '/bin/bash',
 'SHLVL': '1',
 'PATH': '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/hadoop/bin:/opt/hadoop/sbin',
 '_': '/usr/bin/nohup',
 'LANGUAGE': 'en.UTF-8',
 'LANG': 'en.UTF-8',
 'JPY_PARENT_PID': '1566',
 'TERM': 'xterm-color',
 'CLICOLOR': '1',
 'PAGER': 'cat',
 'GIT_PAGER': 'cat',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'JAVA_HOME': '/usr/lib/jvm/java-1.8.0-openjdk-amd64',
 'PDSH_RCMD_TYPE': 'ssh',
 'HADOOP_HOME': '/opt/hadoop',
 'HADOOP_COMMON_HOME': '/opt/hadoop',
 'HADOOP_CONF_DIR': '/opt/hadoop/etc/hadoop',
 'HADOOP_HDFS_HOME': '/opt/hadoop',
 'HADOOP_MAPRED_HOME': '/opt/hadoop',
 'HADOOP_YARN_HOME': '/opt/hadoop'}

# HDFS

## HDFS - Web Interface

- Master node
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
- Worker node
    - hadoop1
        - DataNode: http://localhost:9864
    - hadoop2
        - DataNode: http://localhost:9865
    - hadoop3
        - DataNode: http://localhost:9866

## HDFS - CLI

In [2]:
%%bash

hdfs help

Usage: hdfs [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]

  OPTIONS is none or any of:

--buildpaths                       attempt to add class files from build tree
--config dir                       Hadoop config directory
--daemon (start|status|stop)       operate on a daemon
--debug                            turn on shell script debug mode
--help                             usage information
--hostnames list[,of,host,names]   hosts to use in worker mode
--hosts filename                   list of hosts to use in worker mode
--loglevel level                   set the log4j level for this command
--workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

cacheadmin           configure the HDFS cache
crypto               configure HDFS encryption zones
debug                run a Debug Admin to execute HDFS debug commands
dfsadmin             run a DFS admin client
dfsrouteradmin       manage Router-based federation
ec                   run a HD

## Filesystem Basic Commands

- https://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-common/FileSystemShell.html

Download books from Gutenberg project (http://www.gutenberg.org/)

- Moby Dick; Or, The Whale by Herman Melville
- Pride and Prejudice by Jane Austen
- Dracula by Bram Stoker

In [3]:
%%bash

cd /opt/datasets

wget -qc http://www.gutenberg.org/files/2701/2701-0.txt -O mobydick.txt
wget -qc http://www.gutenberg.org/files/1342/1342-0.txt -O prideandprejudice.txt
wget -qc http://www.gutenberg.org/cache/epub/345/pg345.txt -O dracula.txt

ls /opt/datasets

books
departments.csv
donquixote-output.txt
donquixote.txt
dracula.txt
employees.csv
mobydick.txt
prideandprejudice.txt
shakespeare-output.txt
shakespeare.txt
stations.csv
stop-word-list.csv
trips.csv
ulysses-output.txt
ulysses.txt
weblog.csv


In [ ]:
%%bash

cd /opt/datasets

# create gutenberg folder in HDFS
# hdfs dfs -mkdir /user/hadoop/gutenberg

# copy books to HDFS
# hdfs dfs -put * /user/hadoop/gutenberg
# hdfs dfs -copyFromLocal * /user/hadoop/gutenberg

# list files in HDFS
# hdfs dfs -ls /user/hadoop/gutenberg

# show first KB of file
# hdfs dfs -head /user/hadoop/gutenberg/mobydick.txt

# show last KB of file
# hdfs dfs -tail /user/hadoop/gutenberg/prideandprejudice.txt

# show whole file - CAREFUL
# hdfs dfs -cat /user/hadoop/gutenberg/dracula.txt

# append file contents to a file in HDFS
# hdfs dfs -appendToFile mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/allbooks.txt

# copy allbooks.txt (in HDFS) to gutenberg directory (in HDFS)
# hdfs dfs -cp allbooks.txt /user/hadoop/gutenberg
# hdfs dfs -ls -h -R

# retrieve allbooks.txt from HDFS
# hdfs dfs -get allbooks.txt .
# hdfs dfs -copyToLocal /user/hadoop/allbooks.txt .

# remove file
# hdfs dfs -rm allbooks.txt
# hdfs dfs -rm /user/hadoop/allbooks.txt

# mv file (also used for renaming)
# hdfs dfs -mv gutenberg/allbooks.txt gutenberg/books.txt

# print statistics on folder
# printf "name\ttype\tsize\treps\n"
# hdfs dfs -stat "%n %F %b %r" /user/hadoop/gutenberg/*

# getmerge
# hdfs dfs -getmerge /user/hadoop/gutenberg mergebooks.txt

# remove directory and files (-R recursive)
# hdfs dfs -rm -R /user/hadoop/gutenberg

## Utilization in a MapReduce job

In [4]:
%%bash

cd /opt/datasets

hdfs dfs -mkdir /user/hadoop/gutenberg
hdfs dfs -put mobydick.txt prideandprejudice.txt dracula.txt /user/hadoop/gutenberg

2021-01-28 17:47:36,916 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:47:37,414 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:47:37,582 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [5]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application
hadoop jar ./hadoop-mapreduce-examples-3.2.1.jar wordcount \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

2021-01-28 17:48:14,158 INFO client.RMProxy: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2021-01-28 17:48:14,741 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2021-01-28 17:48:15,402 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1611844877680_0004
2021-01-28 17:48:15,711 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:48:16,199 INFO input.FileInputFormat: Total input files to process : 3
2021-01-28 17:48:16,304 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:48:16,420 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:48:16,473 INFO mapreduce.JobSubmitter: number of splits:3
2021-01-28 17:48:16,797 INFO sasl.SaslDataTransferCl

In [6]:
%%bash

# list output folder contents
hdfs dfs -ls /user/hadoop/gutenberg-output

Found 2 items
-rw-r--r--   2 hadoop hadoop          0 2021-01-28 17:49 /user/hadoop/gutenberg-output/_SUCCESS
-rw-r--r--   2 hadoop hadoop     566225 2021-01-28 17:49 /user/hadoop/gutenberg-output/part-r-00000


In [7]:
%%bash

# show head
hdfs dfs -head /user/hadoop/gutenberg-output/part-r-00000

"'Are	1
"'E's	1
"'I	1
"'Ittin'	1
"'Little	1
"'Lucy,	1
"'Maybe	1
"'Miss	1
"'My	2
"'Never	1
"'No'	1
"'Ow	1
"'Silence!	1
"'That's	1
"'Tyke	1
"'Wilhelmina'--I	1
"'Yes,	1
"A	8
"ABRAHAM	1
"ART."	1
"ARTHUR."	1
"About	1
"Afraid	1
"Again	1
"Agreed!"	1
"Ah	2
"Ah,	16
"Aha!	1
"Aha!"	2
"Alas!	1
"All	7
"Already?"	1
"Am	2
"Amen"	1
"An	1
"An'	1
"And	49
"And,	1
"Answer	1
"Any	1
"Arabian	1
"Are	5
"Arthur	1
"Arthur!	2
"As	6
"Ask	1
"At	3
"Ay,	1
"Back,	1
"Be	1
"Because	7
"Because,	1
"Because,"	2
"Before	1
"Believe	3
"Besides,	1
"Bloofer	1
"Blow	1
"Blue"	1
"Bother	1
"Brave	2
"Bring	1
"But	21
"But,	6
"But,"	4
"By	3
"Call	1
"Can	3
"Can't	1
"Certainly	2
"Certainly,"	1
"Certainly."	1
"Charcot	1
"Come	5
"Come!"	3
"Come,	6
"Come,"	4
"Come.	1
"Count	4
"DEMETER."	1
"DRACULA."	1
"Dear	5
"Defects,"	1
"Denn	1
"Destroyed?"	1
"Did	3
"Do	19
"Doctor,	1
"Don't	3
"Dr.	12
"Draw	1
"Edward	1
"Euthanasia"	1
"Ever	1
"Exactly.	1
"Excuse	1
"FINIS."	1
"Faithfully	1
"Fear	1
"Fifty	2
"Finis,"	1
"For	5
"Forgive	4
"Forgiven!	1
"Frankly

2021-01-28 17:50:37,869 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [8]:
%%bash

cd /tmp

# copy HDFS file to local filesystem
hdfs dfs -get /user/hadoop/gutenberg-output/part-r-00000 gutenberg-output.txt
head /tmp/gutenberg-output.txt

"'Are	1
"'E's	1
"'I	1
"'Ittin'	1
"'Little	1
"'Lucy,	1
"'Maybe	1
"'Miss	1
"'My	2
"'Never	1


2021-01-28 17:51:01,299 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [9]:
%%bash

# remove folder on HDFS
hdfs dfs -rm -R /user/hadoop/gutenberg-output

Deleted /user/hadoop/gutenberg-output


In [10]:
%%bash

cd /opt/hadoop/share/hadoop/mapreduce

# run wordcount application with 2 reducers
hadoop jar ./hadoop-mapreduce-examples-3.2.1.jar wordcount \
-Dmapreduce.job.reduces=2 \
/user/hadoop/gutenberg /user/hadoop/gutenberg-output

2021-01-28 17:51:34,437 INFO client.RMProxy: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2021-01-28 17:51:34,974 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2021-01-28 17:51:35,654 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1611844877680_0005
2021-01-28 17:51:35,945 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:51:36,386 INFO input.FileInputFormat: Total input files to process : 3
2021-01-28 17:51:36,499 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:51:36,623 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:51:36,670 INFO mapreduce.JobSubmitter: number of splits:3
2021-01-28 17:51:37,132 INFO sasl.SaslDataTransferCl

In [11]:
%%bash

# list output folder contents
hdfs dfs -ls /user/hadoop/gutenberg-output

Found 3 items
-rw-r--r--   2 hadoop hadoop          0 2021-01-28 17:52 /user/hadoop/gutenberg-output/_SUCCESS
-rw-r--r--   2 hadoop hadoop     282621 2021-01-28 17:52 /user/hadoop/gutenberg-output/part-r-00000
-rw-r--r--   2 hadoop hadoop     283604 2021-01-28 17:52 /user/hadoop/gutenberg-output/part-r-00001


In [12]:
%%bash

cd /tmp

# copy HDFS file to local filesystem
hdfs dfs -getmerge /user/hadoop/gutenberg-output gutenberg-output.txt
head /tmp/gutenberg-output.txt

hdfs dfs -rm -R /user/hadoop/gutenberg-output

"'Are	1
"'Little	1
"'Maybe	1
"'Miss	1
"'My	2
"'Never	1
"'No'	1
"'Ow	1
"'Silence!	1
"'Wilhelmina'--I	1
Deleted /user/hadoop/gutenberg-output


2021-01-28 17:53:34,013 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2021-01-28 17:53:34,306 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


## Advanced Commands

- https://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-hdfs/HDFSCommands.html

### Verify HDFS cluster status

In [13]:
%%bash

# print topology
hdfs dfsadmin -printTopology

printf "\n%40s\n\n" |tr " " "="

hdfs dfsadmin -report

Rack: /default-rack
   172.17.0.3:9866 (hadoop1)
   172.17.0.4:9866 (hadoop2)
   172.17.0.5:9866 (hadoop3)



Configured Capacity: 303576084480 (282.73 GB)
Present Capacity: 258379335828 (240.63 GB)
DFS Remaining: 258049622016 (240.33 GB)
DFS Used: 329713812 (314.44 MB)
DFS Used%: 0.13%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 101192028160 (94.24 GB)
DFS Used: 164403557 (156.79 MB)
Non DFS Used: 9840400027 (9.16 G

### Replication factor

In [18]:
%%bash

# list folder block location
#hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

# change replication factor of all files in directory to 3
#hdfs dfs -setrep 3 /user/hadoop/gutenberg

# list folder block location
#hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

# change replication factor back to 2
#hdfs dfs -setrep 2 /user/hadoop/gutenberg

# list folder block location
hdfs fsck /user/hadoop/gutenberg -files -blocks -locations

FSCK started by hadoop (auth:SIMPLE) from /172.17.0.2 for path /user/hadoop/gutenberg at Thu Jan 28 17:57:24 GMT 2021
/user/hadoop/gutenberg <dir>
/user/hadoop/gutenberg/dracula.txt 883160 bytes, replicated: replication=2, 1 block(s):  OK
0. BP-1491992694-172.17.0.2-1611844661220:blk_1073741883_1059 len=883160 Live_repl=2  [DatanodeInfoWithStorage[172.17.0.5:9866,DS-2665ec71-1ca6-48d0-9a9f-73461bcfe975,DISK], DatanodeInfoWithStorage[172.17.0.3:9866,DS-c9690db1-9278-47f4-b1b0-94e81a06dfb8,DISK]]

/user/hadoop/gutenberg/mobydick.txt 1276201 bytes, replicated: replication=2, 1 block(s):  OK
0. BP-1491992694-172.17.0.2-1611844661220:blk_1073741881_1057 len=1276201 Live_repl=2  [DatanodeInfoWithStorage[172.17.0.5:9866,DS-2665ec71-1ca6-48d0-9a9f-73461bcfe975,DISK], DatanodeInfoWithStorage[172.17.0.4:9866,DS-fa3ecc8f-ca79-4a02-b2be-62d158883402,DISK]]

/user/hadoop/gutenberg/prideandprejudice.txt 799738 bytes, replicated: replication=2, 1 block(s):  OK
0. BP-1491992694-172.17.0.2-161184466122

Connecting to namenode via http://hadoop:9870/fsck?ugi=hadoop&files=1&blocks=1&locations=1&path=%2Fuser%2Fhadoop%2Fgutenberg


### Decomission nodes

- dfs.hosts.exclude in hdfs-site.xml

In [19]:
%%bash

# Decomissioning hadoop1
cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
hadoop1
EOF

hdfs dfsadmin -refreshNodes

Refresh nodes successful


http://localhost:9870

In [20]:
%%bash

# report HDFS status
hdfs dfsadmin -report

Configured Capacity: 202384056320 (188.48 GB)
Present Capacity: 172037928158 (160.22 GB)
DFS Remaining: 171708358656 (159.92 GB)
DFS Used: 329569502 (314.30 MB)
DFS Used%: 0.19%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Decommissioned
Configured Capacity: 101192028160 (94.24 GB)
DFS Used: 162344960 (154.82 MB)
Non DFS Used: 10004819968 (9.32 GB)
DFS Remaining: 85854179328 (79.96 GB)
DFS Used%: 0.16%
DFS Remaining%: 84.84%
Configured Cache Cap

In [21]:
%%bash

# Recomission all nodes
cat > /opt/hadoop/etc/hadoop/dfs.exclude << EOF
EOF

hdfs dfsadmin -refreshNodes

Refresh nodes successful


In [22]:
%%bash

# report HDFS status
hdfs dfsadmin -report

Configured Capacity: 303576084480 (282.73 GB)
Present Capacity: 258379124364 (240.63 GB)
DFS Remaining: 257995812864 (240.28 GB)
DFS Used: 383311500 (365.55 MB)
DFS Used%: 0.15%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 101192028160 (94.24 GB)
DFS Used: 89298015 (85.16 MB)
Non DFS Used: 9915575201 (9.23 GB)
DFS Remaining: 86016471040 (80.11 GB)
DFS Used%: 0.09%
DFS Remaining%: 85.00%
Configured Cache Capacity: 0 (0

### Handling datanode failures

- timeouts defined in hdfs-site.xml 
    - dfs.namenode.heartbeat.recheck-interval = 10000 (10 seconds)
    - dfs.heartbeat.interval = 3 seconds
- timeout = 2 x recheck-interval + 10 x heartbeat.interval
    - timeout = 50 seconds

In [23]:
%%bash

# get dfs.namenode.heartbeat.recheck-interval
hdfs getconf -confKey dfs.namenode.heartbeat.recheck-interval

# get dfs.heartbeat.interval
hdfs getconf -confKey dfs.heartbeat.interval

10000
3s


In [24]:
%%bash

# simulate node fault
ssh hadoop1 'kill -9 $(cat /tmp/hadoop-hadoop-datanode.pid)'

http://localhost:9870

In [25]:
%%bash

hdfs dfsadmin -report

Configured Capacity: 202384056320 (188.48 GB)
Present Capacity: 172184134227 (160.36 GB)
DFS Remaining: 171854503936 (160.05 GB)
DFS Used: 329630291 (314.36 MB)
DFS Used%: 0.19%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.17.0.4:9866 (hadoop2)
Hostname: hadoop2
Decommission Status : Normal
Configured Capacity: 101192028160 (94.24 GB)
DFS Used: 164804514 (157.17 MB)
Non DFS Used: 9929287774 (9.25 GB)
DFS Remaining: 85927251968 (80.03 GB)
DFS Used%: 0.16%
DFS Remaining%: 84.92%
Configured Cache Capacity: 0 

In [26]:
%%bash

# Restart nodemanager
ssh hadoop1 /opt/hadoop/bin/hdfs --daemon start datanode

In [27]:
%%bash

hdfs dfsadmin -report

Configured Capacity: 303576084480 (282.73 GB)
Present Capacity: 258378976560 (240.63 GB)
DFS Remaining: 258049179648 (240.33 GB)
DFS Used: 329796912 (314.52 MB)
DFS Used%: 0.13%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.17.0.3:9866 (hadoop1)
Hostname: hadoop1
Decommission Status : Normal
Configured Capacity: 101192028160 (94.24 GB)
DFS Used: 89231360 (85.10 MB)
Non DFS Used: 9915719680 (9.23 GB)
DFS Remaining: 86016393216 (80.11 GB)
DFS Used%: 0.09%
DFS Remaining%: 85.00%
Configured Cache Capacity: 0 (0